# Применение парафрейзера rugpt3large_based_on_gpt2
# на параллельном корпусе парафраз RuAdapt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/zlatoust_sentence_aligned_with_CATS.csv')
# df

,source,cos_sim,target,level
0,Ну что ж.,1.000000,Ну что ж.,b2
1,"— Ах, только не мужем, с простою усмешкой сказ...",0.863004,"— Ах, только не мужем, — с простою усмешкой ск...",b1
2,"Мне скоро шестьдесят, я старик, одинокий, ничт...",0.653318,"Ничего во мне нет хорошего, кроме этой любви к...",b2
3,Да!,0.093739,Обстановочка-то какова!,a2_b1
4,"Я вполне уверен, что вы раскаялись и раскаивае...",0.621839,"Я уверен, что вы раскаялись и будете содейство...",b1
...,...,...,...,...
24227,У нас в феврале масленица,0.960889,У нас в феврале масленица.,b1_c1
24228,— Разводиться мне или нет?,0.962737,— Разводиться мне или нет?,b1
24229,"Парусиновую куртку Петр Алексеевич сбросил, ру...",0.938603,"Парусиновую куртку Пётр Алексеевич сбросил, р...",b1_b2
24230,Ай!,1.000000,Ай!,b2


In [ ]:
import pandas as pd
new_df = pd.read_csv('/content/drive/MyDrive/2000_random_rows.csv')
new_df

,Unnamed: 0,source,cos_sim,target
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн..."
...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч..."
1996,1225,— спросил он,0.568087,— спросил Сергей
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в..."


In [ ]:
!pip install git+https://github.com/RussianNLP/russian_paraphrasers@master

In [ ]:
from russian_paraphrasers import GPTParaphraser
paraphraser = GPTParaphraser(model_name="gpt3", range_cand=False, make_eval=False)

In [ ]:
def get_paraphrase(text):
  paraphrase = paraphraser.generate(
      text, n=1, temperature=1, 
      top_k=10, top_p=0.9, 
      max_length=len(text)+10, repetition_penalty=1.5,
      threshold=0.7
    ) 
  return paraphrase['results'][0]['predictions'][0]

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
new_df['output'] = new_df['source'].apply(get_paraphrase)

CPU times: user 58min 39s, sys: 6.8 s, total: 58min 45s
Wall time: 58min 45s


In [ ]:
# Конвертация в csv-файл вместе с output
# new_df.to_csv("result_gpt3.csv")

# Оценка результата парафрейзера rugpt3large_based_on_gpt2
# с помощью дистрибутивной модели rubert-base-cased-sentence

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
import gc
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

In [ ]:
new_df

,Unnamed: 0,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
%%time
DF_FILE = '/content/drive/MyDrive/result_gpt3.csv'
# размер одного обрабатываемого кусочка датафрейма
CHUNKSIZE = 20

def process_frame(new_df):
  # функция принимает на вход датафрейм и возвращает косинусные подобия для каждого ряда
  similarities_3 = []
  for n, row in tqdm(new_df.iterrows()):
    s = row['source']
    o = row['output']
    tokens_s = tokenizer([s], return_tensors="pt", padding=True)
    tokens_o = tokenizer([o], return_tensors="pt", padding=True)
    out_s = model(**tokens_s)
    out_o = model(**tokens_o)
    vec_s = torch.mean(out_s.last_hidden_state, dim=1)
    vec_o = torch.mean(out_o.last_hidden_state, dim=1)
    cos_sim = cos(vec_s, vec_o)
    # item() превращает тензор во float
    similarities_3.append(cos_sim.item())
    # на всякий случай собираем ненужное при помощи garbage collector
    gc.collect()
  return similarities_3

# читаем файл по кусочкам заданного размера
reader = pd.read_csv(DF_FILE, chunksize=CHUNKSIZE)
# количество процессов, которые могут идти одновременно
pool = mp.Pool(4)

funclist_3 = []
for new_df_3 in reader:
  # открываем и обрабатываем каждый кусочек
  new_df.columns = ['Unnamed: 0', 'source', 'cos_sim', 'target', 'output']
  f = pool.apply_async(process_frame, [new_df_3])
  funclist_3.append(f)

result_3 = []
for f in funclist_3:
  # таймаут можно увеличить, если возникает ошибка
  sims = f.get(timeout=500)
  result_3.extend(sims)

20it [00:51,  2.59s/it]
20it [00:52,  2.61s/it]
20it [00:52,  2.62s/it]
20it [00:55,  2.77s/it]
20it [00:43,  2.19s/it]
20it [00:48,  2.43s/it]
20it [00:47,  2.39s/it]
20it [00:51,  2.58s/it]
20it [00:32,  1.62s/it]
20it [00:30,  1.54s/it]
20it [00:28,  1.41s/it]
20it [00:30,  1.55s/it]
20it [00:26,  1.33s/it]
20it [00:23,  1.20s/it]
20it [00:24,  1.21s/it]
20it [00:26,  1.31s/it]
20it [00:26,  1.33s/it]
20it [00:27,  1.39s/it]
20it [00:29,  1.47s/it]
20it [00:26,  1.30s/it]
20it [00:21,  1.09s/it]
20it [00:26,  1.33s/it]
20it [00:23,  1.17s/it]
20it [00:24,  1.25s/it]
20it [00:23,  1.16s/it]
20it [00:22,  1.12s/it]
20it [00:22,  1.11s/it]
20it [00:24,  1.20s/it]
20it [00:26,  1.30s/it]
20it [00:23,  1.20s/it]
20it [00:22,  1.14s/it]
20it [00:25,  1.26s/it]
20it [00:22,  1.13s/it]
20it [00:24,  1.21s/it]
20it [00:25,  1.26s/it]
20it [00:24,  1.23s/it]
20it [00:23,  1.17s/it]
20it [00:22,  1.12s/it]
20it [00:23,  1.19s/it]
20it [00:24,  1.21s/it]
20it [00:22,  1.14s/it]
20it [00:22,  1.

CPU times: user 7.67 s, sys: 1.46 s, total: 9.13 s
Wall time: 11min 12s


In [ ]:
result_3[:2000]

[0.8442684412002563,
 0.7692915201187134,
 0.7344818115234375,
 0.7297821640968323,
 0.7987071871757507,
 0.8327337503433228,
 0.877425491809845,
 0.7285300493240356,
 0.7173037528991699,
 0.8057906627655029,
 0.7046844959259033,
 0.8228777647018433,
 0.7941306829452515,
 0.7984944581985474,
 0.7595106363296509,
 0.8353565335273743,
 0.7445594668388367,
 0.6929668188095093,
 0.7277717590332031,
 0.5606725811958313,
 0.8511689305305481,
 0.5378194451332092,
 0.7501989006996155,
 0.7763757109642029,
 0.8036050796508789,
 0.5792353749275208,
 0.7303248047828674,
 0.5109252333641052,
 0.8081009387969971,
 0.7033427357673645,
 0.8668164610862732,
 0.683777928352356,
 0.5535622835159302,
 0.7014358639717102,
 0.6371468305587769,
 0.6928593516349792,
 0.7697099447250366,
 0.7499057650566101,
 0.6382074356079102,
 0.7152925729751587,
 0.8335355520248413,
 0.733492910861969,
 0.46483147144317627,
 0.6258583068847656,
 0.8302614688873291,
 0.8249168395996094,
 0.7465986013412476,
 0.697612047195

In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(result_3))

0.7101088891848921

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(result_3))

0.11799472058472169

# METEOR
https://huggingface.co/spaces/evaluate-metric/meteor

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

meteor = load("meteor")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1996,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
predictions = final_df['output'].tolist()

In [ ]:
references = final_df['source'].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.06389977479047705}

# BLEU
https://huggingface.co/spaces/evaluate-metric/bleu

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

bleu = load("bleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Мэри это законная жена Джона Локка.', '«Я больше никогда ее снова увижу потому что мы уже будем далеко друг от друга в этой странной новой жизни которую нам предстоит начать сначала для самих себя» (с)...', 'Два старших брата в рассказе о белке спрыгнули при виде приглашавшего их отца без малейшего колебания.', 'Все случилось очень быстро офицер действовал почти автоматически все эти люди были как бы сами по себе каждый из них отдельно от другого их соединяло только общее возбуждение происходящее вокруг дамы с горноспасателями.', 'Отродь никогда в жизни этим делом таким меня ни разу даже на порог дома не впустили а тут сразу кабинетик да кресло кожаное директорское с высокой спинкой напротив окна моего кабинета потому что именно здесь моя работа заключается главным образом чтобы решать поставленные задачи вот как сейчас передо мной поставленную задачу какую по счету но самую главную мою работу над заключительной часть', 'Разве вы его совсем забыли или это было так давно и он умер вов

In [ ]:
bleu_results = bleu.compute(predictions=predictions, references=references)
print(bleu_results)

{'bleu': 0.0013317246445428805, 'precisions': [0.07092980521866961, 0.0019900822132324155, 0.00027921262041044257, 0.00011254079603856398], 'brevity_penalty': 0.9176524609020204, 'length_ratio': 0.9208641209318067, 'translation_length': 32652, 'reference_length': 35458}


# SacreBLEU
https://huggingface.co/spaces/evaluate-metric/sacrebleu

In [ ]:
pip install evaluate

In [ ]:
pip install sacrebleu

In [ ]:
from evaluate import load

sacrebleu = load("sacrebleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Мэри это законная жена Джона Локка.', '«Я больше никогда ее снова увижу потому что мы уже будем далеко друг от друга в этой странной новой жизни которую нам предстоит начать сначала для самих себя» (с)...', 'Два старших брата в рассказе о белке спрыгнули при виде приглашавшего их отца без малейшего колебания.', 'Все случилось очень быстро офицер действовал почти автоматически все эти люди были как бы сами по себе каждый из них отдельно от другого их соединяло только общее возбуждение происходящее вокруг дамы с горноспасателями.', 'Отродь никогда в жизни этим делом таким меня ни разу даже на порог дома не впустили а тут сразу кабинетик да кресло кожаное директорское с высокой спинкой напротив окна моего кабинета потому что именно здесь моя работа заключается главным образом чтобы решать поставленные задачи вот как сейчас передо мной поставленную задачу какую по счету но самую главную мою работу над заключительной часть', 'Разве вы его совсем забыли или это было так давно и он умер вов

In [ ]:
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(sacrebleu_results)
print(round(sacrebleu_results["score"], 1))

{'score': 0.13317246445428796, 'counts': [2316, 61, 8, 3], 'totals': [32652, 30652, 28652, 26657], 'precisions': [7.092980521866961, 0.19900822132324156, 0.027921262041044254, 0.011254079603856397], 'bp': 0.9176524609020204, 'sys_len': 32652, 'ref_len': 35458}
0.1


# ROUGE
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Мэри это законная жена Джона Локка.', '«Я больше никогда ее снова увижу потому что мы уже будем далеко друг от друга в этой странной новой жизни которую нам предстоит начать сначала для самих себя» (с)...', 'Два старших брата в рассказе о белке спрыгнули при виде приглашавшего их отца без малейшего колебания.', 'Все случилось очень быстро офицер действовал почти автоматически все эти люди были как бы сами по себе каждый из них отдельно от другого их соединяло только общее возбуждение происходящее вокруг дамы с горноспасателями.', 'Отродь никогда в жизни этим делом таким меня ни разу даже на порог дома не впустили а тут сразу кабинетик да кресло кожаное директорское с высокой спинкой напротив окна моего кабинета потому что именно здесь моя работа заключается главным образом чтобы решать поставленные задачи вот как сейчас передо мной поставленную задачу какую по счету но самую главную мою работу над заключительной часть', 'Разве вы его совсем забыли или это было так давно и он умер вов

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split())
rouge_results

{'rouge1': 0.03216519751074422,
 'rouge2': 0.000402534965034965,
 'rougeL': 0.03216629887168962,
 'rougeLsum': 0.032182157685841865}

# BERTScore
https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Мэри это законная жена Джона Локка.', '«Я больше никогда ее снова увижу потому что мы уже будем далеко друг от друга в этой странной новой жизни которую нам предстоит начать сначала для самих себя» (с)...', 'Два старших брата в рассказе о белке спрыгнули при виде приглашавшего их отца без малейшего колебания.', 'Все случилось очень быстро офицер действовал почти автоматически все эти люди были как бы сами по себе каждый из них отдельно от другого их соединяло только общее возбуждение происходящее вокруг дамы с горноспасателями.', 'Отродь никогда в жизни этим делом таким меня ни разу даже на порог дома не впустили а тут сразу кабинетик да кресло кожаное директорское с высокой спинкой напротив окна моего кабинета потому что именно здесь моя работа заключается главным образом чтобы решать поставленные задачи вот как сейчас передо мной поставленную задачу какую по счету но самую главную мою работу над заключительной часть', 'Разве вы его совсем забыли или это было так давно и он умер вов

In [ ]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results

{'precision': [0.8866748809814453,
  0.8557377457618713,
  0.8871606588363647,
  0.8940075039863586,
  0.8148373365402222,
  0.8403201103210449,
  0.8812525272369385,
  0.8451652526855469,
  0.8410325050354004,
  0.8444690704345703,
  0.8431074619293213,
  0.8786134123802185,
  0.8750878572463989,
  0.8995028734207153,
  0.8218526840209961,
  0.8289613127708435,
  0.8661316633224487,
  0.7917813658714294,
  0.8896583318710327,
  0.8099560737609863,
  0.8743883371353149,
  0.8101444244384766,
  0.8882768750190735,
  0.8676882386207581,
  0.8609503507614136,
  0.8824089765548706,
  0.8792755603790283,
  0.8328669667243958,
  0.8882046937942505,
  0.837926983833313,
  0.859509289264679,
  0.862142026424408,
  0.8529129028320312,
  0.736082911491394,
  0.8906652331352234,
  0.8533815145492554,
  0.8570992350578308,
  0.7844899892807007,
  0.8472380638122559,
  0.8671144247055054,
  0.848590075969696,
  0.8564785122871399,
  0.8008915781974792,
  0.8597890734672546,
  0.8755947351455688,
  

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['precision']))

0.859615243524313

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['recall']))

0.8649986435770989

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['f1']))

0.8620002340674401

# ParaScore
https://github.com/shadowkiller33/ParaScore

In [ ]:
pip install parascore==1.0.5

In [ ]:
from parascore import ParaScorer

In [ ]:
scorer = ParaScorer(lang="ru", model_type='bert-base-uncased')

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt3.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,Мэри это законная жена Джона Локка.
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",«Я больше никогда ее снова увижу потому что мы...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Два старших брата в рассказе о белке спрыгнули...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все случилось очень быстро офицер действовал п...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",Отродь никогда в жизни этим делом таким меня н...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Он поехал ночью на вокзал провожать свою девуш...
1996,1225,— спросил он,0.568087,— спросил Сергей,Ты не хочешь спросить меня о том же самом дружищ.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,США нанесут удар первыми.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",Хорошо что ты сам явился ко мне с этим вопросо...


In [ ]:
cands = final_df['output'].tolist()
sources = final_df['source'].tolist()
refs = final_df['target'].tolist()

In [ ]:
parascore_results = scorer.base_score(cands, sources, refs, batch_size=16)
parascore_results

[0.7123800086975097,
 0.7182807493209838,
 0.7398591804504394,
 0.7512188720703125,
 0.664591805934906,
 0.669961290359497,
 0.7099550533294677,
 0.6567630934715271,
 0.6492161917686462,
 0.7132709789276123,
 0.703296856880188,
 0.7097964453697204,
 0.7431995439529419,
 0.7716958689689636,
 0.7080938982963562,
 0.6484288740158081,
 0.7146023082733154,
 0.6675493288040161,
 0.759372489452362,
 0.6528234052658081,
 0.7159419226646423,
 0.6322294640541076,
 0.7519186305999755,
 0.7210214900970459,
 0.6727190780639648,
 0.7300750184059143,
 0.719451801776886,
 0.664856629371643,
 0.7227080035209655,
 0.6924386191368103,
 0.7317930030822754,
 0.7052269744873046,
 0.7384222316741943,
 0.6077208089828491,
 0.7380783724784851,
 0.6974290776252746,
 0.6940151023864746,
 0.6589519548416137,
 0.6779803204536438,
 0.7081654238700866,
 0.6620132493972778,
 0.6776507067680358,
 0.6413847970962524,
 0.641551809310913,
 0.735252456665039,
 0.7148581910133361,
 0.6857685613632202,
 0.7315123844146728,


In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(parascore_results))

0.7108090316832179

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(parascore_results))

0.050384525217307456